# Dfsu 3D

Layered dfsu files comes in several different shapes:

* 3D
* 2D Vertical slice (transect)
* 1D Vertical profile

Two layer systems exist:

* sigma (terrain and surface following coordinates)
* sigma-z (sigma layers at the top and fixed z-layers at the bottom)

In sigma-layered files, all columns has the same number of layers. In sigma-z files, the number of z-layers can be different for different columns.  

Layered dfsu files have a "hidden" first dynamic item called "zn" with the (dynamic) z-positions of the nodes. 

Read the [MIKE IO dfsu-3d documentation](https://dhi.github.io/mikeio/dfsu-3d.html) for more info.

In [ ]:
import matplotlib.pyplot as plt
import mikeio

## 3D Sigma-z

In [ ]:
filename = "data/oresund_sigma_z.dfsu"
dfs = mikeio.open(filename)
dfs

Apart from the normal dfsu properties, layered dfsu files have these properties: 

In [ ]:
print(f"Maximum number of layers: {dfs.n_layers}")
print(f"Number of sigma layers: {dfs.n_sigma_layers}")
print(f"Maximum number of z-layers: {dfs.n_z_layers}")
print(f"The layer number for each 3d element: {dfs.layer_ids}")
print(f"List of 3d element ids of surface layer: {dfs.top_elements}")
print(f"List of 3d element ids of bottom layer: {dfs.bottom_elements}")
print(f"List of number of layers for each column: {dfs.n_layers_per_column}")
print(f"The 2d-to-3d element connectivity table for a 3d object: {dfs.e2_e3_table[:3]} ...")
print(f"The associated 2d element id for each 3d element: {dfs.elem2d_ids}")

The associated 2D geometry for a 3D file can be outputted in this way:

In [ ]:
geom2d = dfs.geometry2d
geom2d

In [ ]:
geom2d.n_elements

### Layers in a 3D file

Get element ids for a specific layer with the get_layer_elements() method. Layer ids are 0-based (new in MIKE IO 0.10). 

In [ ]:
print("Number of elements per layer (5 z-layers, 4 sigma layers):")
for j in range(dfs.n_layers):
    print(f"Layer {j}: {len(dfs.get_layer_elements(j))}")

In [ ]:
# the bottom layer is a list of the elements with the lowest id for each column
# For sigma-z files it is NOT the same as layer 1
print(f"Bottom layer: {len(dfs.bottom_elements)}")

### Surface layer of 3D file

In [ ]:
ds = dfs.read(layers="top")
print(ds)

In [ ]:
ds["Temperature"].plot();

In [ ]:
max_t = ds['Temperature'].values.max()
print(f'Maximum temperature in top layer: {max_t:.1f}C')

### Find position of max temperature and plot

Use numpy argmax() method to find the element with the largest value.

In [ ]:
timestep = 0
max_elem_id = ds['Temperature'].isel(time=timestep).values.argmax()
top_element_coordinates = dfs.element_coordinates[dfs.top_elements]
max_x, max_y = top_element_coordinates[max_elem_id][:2]
max_x, max_y

In [ ]:
ax = ds['Temperature'].plot(figsize=(6,7))
ax.plot(max_x, max_y, marker='*', markersize=20);

# Read 1D profile from 3D file

Find water column which has highest temperature and plot profile for all 3 time steps using dynamic z information. 

In [ ]:
dsp = dfs.read(x=max_x, y=max_y) # select vertical column from dfsu-3d 

In [ ]:
dsp['Temperature'].shape   # 3 timesteps and 4 layers (no z-layers at this position)

In [ ]:
dsp['Temperature'].plot();

## Read vertical slice

In [ ]:
filename = "data/oresund_vertical_slice.dfsu"
dfs = mikeio.open(filename)
dfs

In [ ]:
print(dfs.bottom_elements[:9])
print(dfs.n_layers_per_column[:9])
print(dfs.top_elements[:9])

In [ ]:
da = dfs.read(items="Temperature")[0]

In [ ]:
da.plot();